In [1]:
from sqlalchemy import Column, DateTime, Integer, String, Boolean, ForeignKey, create_engine, text, Enum
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import scoped_session, sessionmaker, relationship
import os
import datetime
if os.path.exists("orm-test.db"):
    os.remove("orm-test.db")
engine = create_engine('sqlite:///orm-test.db', echo=True, convert_unicode=True)
Base = declarative_base()

In [2]:
class User(Base):
    __tablename__ = 'users'
    id = Column(Integer, primary_key=True)
    username = Column(String(31), unique=True, nullable=False)
    password = Column(String(31))
    name = Column(String(255))
    email = Column(String(255))
    phone = Column(String(255))
    admin = Column(Boolean, default=False)
    investor = Column(Boolean, default=False)
    locale = Column(String(5), default='en_US')
    verified = Column(Boolean, default=False)
    blocked = Column(Boolean, default=False)
    created = Column(DateTime(), server_default=text("(datetime('now'))"))
    updated = Column(DateTime(), server_default=text("(datetime('now'))"))
    bids = relationship("Bid", back_populates="bidder")
    asks = relationship("Ask", back_populates="investor")
    book = relationship("Book", back_populates="owner")
    wallets = relationship("Wallet", back_populates="owner")
    transactions = relationship("Transaction", back_populates="user")
    sells = relationship("Exchange", back_populates="seller", primaryjoin="Exchange.seller_id==User.id")
    purchases = relationship("Exchange", back_populates="buyer", primaryjoin="Exchange.buyer_id==User.id")

    def __repr__(self):
        return "<User(id='%s', username='%s', email='%s', phone='%s', admin='%s', verified='%s', blocked='%s')>" % (
            self.id, self.username, self.email, self.phone, self.admin, self.verified, self.blocked
        )

    def update(self, **kwargs):
        if len(kwargs):
            self.updated = datetime.datetime.utcnow()
            p = ('username', 'password', 'name', 'email', 'phone', 'admin', 'investor', 'locale', 'verified', 'blocked')
            for k, v in kwargs.items():
                if k in p:
                    self.__setattr__(k, v)

    def dump(self):
        return dict([(k, v) for k, v in vars(self).items() if not k.startswith('_')])


class Bid(Base):
    __tablename__ = 'bids'
    id = Column(Integer, primary_key=True)
    symbol = Column(String(15), ForeignKey('companies.id'), nullable=False)
    quantity = Column(Integer)
    price = Column(Integer)
    user_id = Column(Integer, ForeignKey('users.id'), nullable=False)
    active = Column(Boolean, default=True)
    created = Column(DateTime(), server_default=text("(datetime('now'))"))
    updated = Column(DateTime(), server_default=text("(datetime('now'))"))
    bidder = relationship("User", back_populates="bids")
    company = relationship("Company", back_populates="bids")

    def __repr__(self):
        return "<Bid(id='%s', symbol='%s', quantity='%s', price='%s', user_id='%s', active='%s')>" % (
            self.id, self.symbol, self.quantity, self.price, self.user_id, self.active
        )

    def update(self, **kwargs):
        if len(kwargs):
            self.updated = datetime.datetime.utcnow()
            p = ('quantity', 'price', 'bidder', 'status')
            for k, v in kwargs.items():
                if k in p:
                    self.__setattr__(k, v)

    def dump(self):
        return dict([(k, v) for k, v in vars(self).items() if not k.startswith('_')])


class Ask(Base):
    __tablename__ = 'asks'
    id = Column(Integer, primary_key=True)
    symbol = Column(String(15), ForeignKey('companies.id'), nullable=False)
    quantity = Column(Integer)
    price = Column(Integer)
    user_id = Column(Integer, ForeignKey('users.id'), nullable=False)
    active = Column(Boolean, default=True)
    created = Column(DateTime(), server_default=text("(datetime('now'))"))
    updated = Column(DateTime(), server_default=text("(datetime('now'))"))
    investor = relationship("User", back_populates="asks")
    company = relationship("Company", back_populates="asks")

    def __repr__(self):
        return "<Ask(id='%s', symbol='%s', quantity='%s', price='%s', user_id='%s', active='%s')>" % (
            self.id, self.symbol, self.quantity, self.price, self.user_id, self.active
        )

    def update(self, **kwargs):
        if len(kwargs):
            self.updated = datetime.datetime.utcnow()
            p = ('quantity', 'price', 'bidder', 'status')
            for k, v in kwargs.items():
                if k in p:
                    self.__setattr__(k, v)

    def dump(self):
        return dict([(k, v) for k, v in vars(self).items() if not k.startswith('_')])

    
class Book(Base):
    __tablename__ = 'book'
    id = Column(Integer, primary_key=True)
    symbol = Column(String(15), ForeignKey('companies.id'), nullable=False)
    quantity = Column(Integer, nullable=False)
    user_id = Column(Integer, ForeignKey('users.id'), nullable=False)
    created = Column(DateTime(), server_default=text("(datetime('now'))"))
    updated = Column(DateTime(), server_default=text("(datetime('now'))"))
    owner = relationship("User", back_populates="book")
    company = relationship("Company", back_populates="book")

    def __repr__(self):
        return "<Book(id='%s', symbol='%s', quantity='%s', user_id='%s')>" % (
            self.id, self.symbol, self.quantity, self.user_id
        )

    def update(self, **kwargs):
        if len(kwargs):
            self.updated = datetime.datetime.utcnow()
            p = ('quantity')
            for k, v in kwargs.items():
                if k in p:
                    self.__setattr__(k, v)

    def update(self, **kwargs):
        pass

    def dump(self):
        return dict([(k, v) for k, v in vars(self).items() if not k.startswith('_')])    


class Wallet(Base):
    __tablename__ = 'wallets'
    id = Column(Integer, primary_key=True)
    currency = Column(String(3), nullable=False, default='VND')
    amount = Column(Integer, nullable=False)
    user_id = Column(Integer, ForeignKey('users.id'), nullable=False)
    created = Column(DateTime(), server_default=text("(datetime('now'))"))
    updated = Column(DateTime(), server_default=text("(datetime('now'))"))
    owner = relationship("User", back_populates="wallets")

    def __repr__(self):
        return "<Wallet(id='%s', currency='%s', amount='%s', user_id='%s')>" % (
            self.id, self.currency, self.amount, self.user_id
        )

    def update(self, **kwargs):
        if len(kwargs):
            self.updated = datetime.datetime.utcnow()
            p = ('amount')
            for k, v in kwargs.items():
                if k in p:
                    self.__setattr__(k, v)

    def update(self, **kwargs):
        pass

    def dump(self):
        return dict([(k, v) for k, v in vars(self).items() if not k.startswith('_')])    


class Transaction(Base):
    __tablename__ = 'transactions'
    id = Column(Integer, primary_key=True)
    currency = Column(String(3), nullable=False, default='VND')
    sent = Column(Integer, nullable=False)
    received = Column(Integer, nullable=False)
    fee = Column(Integer, nullable=False)
    user_id = Column(Integer, ForeignKey('users.id'), nullable=False)
    description = Column(String)
    tags = Column(String)
    created = Column(DateTime(), server_default=text("(datetime('now'))"))
    user = relationship("User", back_populates="transactions")

    def __repr__(self):
        return "<Transaction(id='%s', currency='%s', sent='%s', received='%s', user_id='%s', description='%s')>" % (
            self.id, self.currency, self.sent, self.received, self.user_id, self.description
        )

    def update(self, **kwargs):
        pass

    def dump(self):
        return dict([(k, v) for k, v in vars(self).items() if not k.startswith('_')])  

    
class Exchange(Base):
    __tablename__ = 'exchanges'
    id = Column(Integer, primary_key=True)
    symbol = Column(String(15), ForeignKey('companies.id'), nullable=False)
    quantity = Column(Integer)
    price = Column(Integer)
    seller_id = Column(Integer, ForeignKey('users.id'), nullable=False)
    buyer_id = Column(Integer, ForeignKey('users.id'), nullable=False)
    created = Column(DateTime(), server_default=text("(datetime('now'))"))
    seller = relationship("User", back_populates="sells", primaryjoin="User.id==Exchange.seller_id")
    buyer = relationship("User", back_populates="purchases", primaryjoin="User.id==Exchange.buyer_id")
    company = relationship("Company", back_populates="exchanges")

    def __repr__(self):
        return "<Exchange(id='%s', symbol='%s', quantity='%s', price='%s', seller_id='%s', buyer_id='%s')>" % (
            self.id, self.symbol, self.quantity, self.price, self.seller_id, self.buyer_id
        )

    def update(self, **kwargs):
        pass

    def dump(self):
        return dict([(k, v) for k, v in vars(self).items() if not k.startswith('_')])  

    
class Company(Base):
    __tablename__ = 'companies'
    id = Column(Integer, primary_key=True)
    symbol = Column(String(15), unique=True, nullable=False)
    company_name = Column(String(255), nullable=False)
    industry = Column(String(255))
    website = Column(String(255))
    description = Column(String)
    sector = Column(String(255))
    tags = Column(String)
    created = Column(DateTime(), server_default=text("(datetime('now'))"))
    updated = Column(DateTime(), server_default=text("(datetime('now'))"))
    bids = relationship("Bid", back_populates="company")
    asks = relationship("Ask", back_populates="company")
    book = relationship("Book", back_populates="company")
    exchanges = relationship("Exchange", back_populates="company")

    def __repr__(self):
        return "<Company(id='%s', symbol='%s', company_name='%s', website='%s')>" % (
            self.id, self.symbol, self.company_name, self.website
        )

    def update(self, **kwargs):
        if len(kwargs):
            self.updated = datetime.datetime.utcnow()
            p = ('symbol', 'company_name', 'industry', 'website', 'description', 'sector', 'tags')
            for k, v in kwargs.items():
                if k in p:
                    self.__setattr__(k, v)

    def dump(self):
        return dict([(k, v) for k, v in vars(self).items() if not k.startswith('_')])

In [3]:
db_session = scoped_session(sessionmaker(autocommit=False, autoflush=False, bind=engine))

In [4]:
Base.query = db_session.query_property()

In [5]:
Base.metadata.create_all(bind=engine)

2018-12-15 00:44:18,213 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2018-12-15 00:44:18,214 INFO sqlalchemy.engine.base.Engine ()
2018-12-15 00:44:18,216 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2018-12-15 00:44:18,216 INFO sqlalchemy.engine.base.Engine ()
2018-12-15 00:44:18,218 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("users")
2018-12-15 00:44:18,220 INFO sqlalchemy.engine.base.Engine ()
2018-12-15 00:44:18,220 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("bids")
2018-12-15 00:44:18,221 INFO sqlalchemy.engine.base.Engine ()
2018-12-15 00:44:18,222 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("asks")
2018-12-15 00:44:18,223 INFO sqlalchemy.engine.base.Engine ()
2018-12-15 00:44:18,225 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("book")
2018-12-15 00:44:18,225 INFO sqlalchemy.engine.base.Engine ()
2018-12-15 00:44:18,226 INFO sqlalchemy.engine.ba

In [6]:
db_session.add(User(username='pericaperic'))
db_session.add(User(username='johndoe'))
db_session.add(User(username='seletelek'))
db_session.commit()

2018-12-15 00:44:18,571 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2018-12-15 00:44:18,574 INFO sqlalchemy.engine.base.Engine INSERT INTO users (username, password, name, email, phone, admin, investor, locale, verified, blocked) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
2018-12-15 00:44:18,575 INFO sqlalchemy.engine.base.Engine ('pericaperic', None, None, None, None, 0, 0, 'en_US', 0, 0)
2018-12-15 00:44:18,577 INFO sqlalchemy.engine.base.Engine INSERT INTO users (username, password, name, email, phone, admin, investor, locale, verified, blocked) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
2018-12-15 00:44:18,578 INFO sqlalchemy.engine.base.Engine ('johndoe', None, None, None, None, 0, 0, 'en_US', 0, 0)
2018-12-15 00:44:18,580 INFO sqlalchemy.engine.base.Engine INSERT INTO users (username, password, name, email, phone, admin, investor, locale, verified, blocked) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
2018-12-15 00:44:18,581 INFO sqlalchemy.engine.base.Engine ('seletelek', None, None, Non

In [7]:
db_session.add(Company(symbol='OLI', company_name='Oliver LTD.'))
db_session.commit()

2018-12-15 00:44:19,457 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2018-12-15 00:44:19,459 INFO sqlalchemy.engine.base.Engine INSERT INTO companies (symbol, company_name, industry, website, description, sector, tags) VALUES (?, ?, ?, ?, ?, ?, ?)
2018-12-15 00:44:19,460 INFO sqlalchemy.engine.base.Engine ('OLI', 'Oliver LTD.', None, None, None, None, None)
2018-12-15 00:44:19,462 INFO sqlalchemy.engine.base.Engine COMMIT


In [8]:
# u1 = db_session.query(User).filter(User.id == 1).one_or_none()
u1, u2, u3 = db_session.query(User).limit(3).all()
(u1, u2, u3)

2018-12-15 00:44:20,207 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2018-12-15 00:44:20,209 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.username AS users_username, users.password AS users_password, users.name AS users_name, users.email AS users_email, users.phone AS users_phone, users.admin AS users_admin, users.investor AS users_investor, users.locale AS users_locale, users.verified AS users_verified, users.blocked AS users_blocked, users.created AS users_created, users.updated AS users_updated 
FROM users
 LIMIT ? OFFSET ?
2018-12-15 00:44:20,210 INFO sqlalchemy.engine.base.Engine (3, 0)


(<User(id='1', username='pericaperic', email='None', phone='None', admin='False', verified='False', blocked='False')>,
 <User(id='2', username='johndoe', email='None', phone='None', admin='False', verified='False', blocked='False')>,
 <User(id='3', username='seletelek', email='None', phone='None', admin='False', verified='False', blocked='False')>)

In [9]:
u1.bids.append(Bid(symbol='OLI', quantity=3,price=3456))

2018-12-15 00:44:21,413 INFO sqlalchemy.engine.base.Engine SELECT bids.id AS bids_id, bids.symbol AS bids_symbol, bids.quantity AS bids_quantity, bids.price AS bids_price, bids.user_id AS bids_user_id, bids.active AS bids_active, bids.created AS bids_created, bids.updated AS bids_updated 
FROM bids 
WHERE ? = bids.user_id
2018-12-15 00:44:21,414 INFO sqlalchemy.engine.base.Engine (1,)


In [10]:
db_session.new

IdentitySet([<Bid(id='None', symbol='OLI', quantity='3', price='3456', user_id='None', active='None')>])

In [11]:
db_session.commit()

2018-12-15 00:44:24,602 INFO sqlalchemy.engine.base.Engine INSERT INTO bids (symbol, quantity, price, user_id, active) VALUES (?, ?, ?, ?, ?)
2018-12-15 00:44:24,603 INFO sqlalchemy.engine.base.Engine ('OLI', 3, 3456, 1, 1)
2018-12-15 00:44:24,605 INFO sqlalchemy.engine.base.Engine COMMIT


In [12]:
u1.bids

2018-12-15 00:44:25,705 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2018-12-15 00:44:25,706 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.username AS users_username, users.password AS users_password, users.name AS users_name, users.email AS users_email, users.phone AS users_phone, users.admin AS users_admin, users.investor AS users_investor, users.locale AS users_locale, users.verified AS users_verified, users.blocked AS users_blocked, users.created AS users_created, users.updated AS users_updated 
FROM users 
WHERE users.id = ?
2018-12-15 00:44:25,707 INFO sqlalchemy.engine.base.Engine (1,)
2018-12-15 00:44:25,709 INFO sqlalchemy.engine.base.Engine SELECT bids.id AS bids_id, bids.symbol AS bids_symbol, bids.quantity AS bids_quantity, bids.price AS bids_price, bids.user_id AS bids_user_id, bids.active AS bids_active, bids.created AS bids_created, bids.updated AS bids_updated 
FROM bids 
WHERE ? = bids.user_id
2018-12-15 00:44:25,710 INFO sqlalchemy.engin

[<Bid(id='1', symbol='OLI', quantity='3', price='3456', user_id='1', active='True')>]

In [13]:
u1.asks.append(Ask(symbol='OLI', quantity=3,price=3456))

2018-12-15 00:44:27,577 INFO sqlalchemy.engine.base.Engine SELECT asks.id AS asks_id, asks.symbol AS asks_symbol, asks.quantity AS asks_quantity, asks.price AS asks_price, asks.user_id AS asks_user_id, asks.active AS asks_active, asks.created AS asks_created, asks.updated AS asks_updated 
FROM asks 
WHERE ? = asks.user_id
2018-12-15 00:44:27,578 INFO sqlalchemy.engine.base.Engine (1,)


In [14]:
db_session.commit()

2018-12-15 00:44:28,069 INFO sqlalchemy.engine.base.Engine INSERT INTO asks (symbol, quantity, price, user_id, active) VALUES (?, ?, ?, ?, ?)
2018-12-15 00:44:28,070 INFO sqlalchemy.engine.base.Engine ('OLI', 3, 3456, 1, 1)
2018-12-15 00:44:28,072 INFO sqlalchemy.engine.base.Engine COMMIT


In [18]:
u1.asks[0].investor

<User(id='1', username='pericaperic', email='None', phone='None', admin='False', verified='False', blocked='False')>

In [19]:
u1.sells.append(Exchange(symbol='OLI', quantity=3, price=3456, buyer_id=2))

In [20]:
db_session.commit()

2018-12-15 00:45:15,484 INFO sqlalchemy.engine.base.Engine INSERT INTO exchanges (symbol, quantity, price, seller_id, buyer_id) VALUES (?, ?, ?, ?, ?)
2018-12-15 00:45:15,485 INFO sqlalchemy.engine.base.Engine ('OLI', 3, 3456, 1, 2)
2018-12-15 00:45:15,487 INFO sqlalchemy.engine.base.Engine INSERT INTO exchanges (symbol, quantity, price, seller_id, buyer_id) VALUES (?, ?, ?, ?, ?)
2018-12-15 00:45:15,487 INFO sqlalchemy.engine.base.Engine ('OLI', 3, 3456, 1, 2)
2018-12-15 00:45:15,488 INFO sqlalchemy.engine.base.Engine COMMIT


In [21]:
u1.sells

2018-12-15 00:45:16,337 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2018-12-15 00:45:16,339 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.username AS users_username, users.password AS users_password, users.name AS users_name, users.email AS users_email, users.phone AS users_phone, users.admin AS users_admin, users.investor AS users_investor, users.locale AS users_locale, users.verified AS users_verified, users.blocked AS users_blocked, users.created AS users_created, users.updated AS users_updated 
FROM users 
WHERE users.id = ?
2018-12-15 00:45:16,340 INFO sqlalchemy.engine.base.Engine (1,)
2018-12-15 00:45:16,341 INFO sqlalchemy.engine.base.Engine SELECT exchanges.id AS exchanges_id, exchanges.symbol AS exchanges_symbol, exchanges.quantity AS exchanges_quantity, exchanges.price AS exchanges_price, exchanges.seller_id AS exchanges_seller_id, exchanges.buyer_id AS exchanges_buyer_id, exchanges.created AS exchanges_created 
FROM exchanges 
WHERE exchanges

[<Exchange(id='1', symbol='OLI', quantity='3', price='3456', seller_id='1', buyer_id='2')>,
 <Exchange(id='2', symbol='OLI', quantity='3', price='3456', seller_id='1', buyer_id='2')>]

In [22]:
u2.purchases[0].seller

2018-12-15 00:45:19,060 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.username AS users_username, users.password AS users_password, users.name AS users_name, users.email AS users_email, users.phone AS users_phone, users.admin AS users_admin, users.investor AS users_investor, users.locale AS users_locale, users.verified AS users_verified, users.blocked AS users_blocked, users.created AS users_created, users.updated AS users_updated 
FROM users 
WHERE users.id = ?
2018-12-15 00:45:19,061 INFO sqlalchemy.engine.base.Engine (2,)
2018-12-15 00:45:19,064 INFO sqlalchemy.engine.base.Engine SELECT exchanges.id AS exchanges_id, exchanges.symbol AS exchanges_symbol, exchanges.quantity AS exchanges_quantity, exchanges.price AS exchanges_price, exchanges.seller_id AS exchanges_seller_id, exchanges.buyer_id AS exchanges_buyer_id, exchanges.created AS exchanges_created 
FROM exchanges 
WHERE exchanges.buyer_id = ?
2018-12-15 00:45:19,065 INFO sqlalchemy.engine.base.Engine (2,

<User(id='1', username='pericaperic', email='None', phone='None', admin='False', verified='False', blocked='False')>

In [23]:
u1.update(email='mr.oliver.nadj@gmail.com')

In [24]:
db_session.commit()

2018-12-15 00:45:25,111 INFO sqlalchemy.engine.base.Engine UPDATE users SET email=?, updated=? WHERE users.id = ?
2018-12-15 00:45:25,112 INFO sqlalchemy.engine.base.Engine ('mr.oliver.nadj@gmail.com', '2018-12-14 17:45:23.949786', 1)
2018-12-15 00:45:25,115 INFO sqlalchemy.engine.base.Engine COMMIT


In [25]:
(u1.updated, u1.created)

2018-12-15 00:45:26,072 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2018-12-15 00:45:26,073 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.username AS users_username, users.password AS users_password, users.name AS users_name, users.email AS users_email, users.phone AS users_phone, users.admin AS users_admin, users.investor AS users_investor, users.locale AS users_locale, users.verified AS users_verified, users.blocked AS users_blocked, users.created AS users_created, users.updated AS users_updated 
FROM users 
WHERE users.id = ?
2018-12-15 00:45:26,074 INFO sqlalchemy.engine.base.Engine (1,)


(datetime.datetime(2018, 12, 14, 17, 45, 23, 949786),
 datetime.datetime(2018, 12, 14, 17, 44, 18))